# Домашнее задание по программированию

В данной домашней работе вам предстоит ознакомиться с задачей классификации и ее двумя решениями: аналитическим и с помощью SGD. Затем вам будет необходимо изменить код так, чтобы он смог решать задачу регрессии (так же двумя способами).

Полезные теоретические материалы:
* [Лекции для студентов МФТИ.](https://github.com/ml-mipt/ml-mipt-part1/blob/master/2018/lectures/Lecture_06_linear_models.pdf) В данной лекции приводится основная часть кода для SGD
* [Презентации с прошлого семинара](https://drive.google.com/drive/u/0/folders/1JI3pMlmiACFn19tSTR_C3XAOT4e37tqp)
* [Здесь](https://onlinecourses.science.psu.edu/stat501/node/250/) можно посмотреть 1 и 2 пункты

Сначала импортируем все необходимые для работы библиотеки и функции.

In [1]:
import numpy as np

from random import randint

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Breast Cancer

Для примера рассмотрим решение задачи классификации больных раком с помощью линейной регресии. Необходимо для каждого человека сказать, доброкачественная или злокачественная у него опухоль.

Загружаем датасет.

In [2]:
cancer = datasets.load_breast_cancer()

# Описание датасета можно вывести с помощью следующей функции
# cancer.DESCR

Добавим столбец из единичек - чтобы формулы легко переписывались в виде операций над матрциами и векторами.

In [3]:
X = np.hstack((np.ones(len(cancer.target)).reshape(-1,1),cancer.data))

Разобьем всю выборку на **обучающую (train)** и **тестовую (test)**. Идея заключается в том, что на обучающей выборке вы, собственно, обучаетесь, то есть подбираете веса вашей модели. На тестовой выборке вы находите ответы вашего алгоритма и сравниваете их с истинными ответами.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, cancer.target, random_state=42)

### Аналитическое решение

Для удобства напишем класс **LinearRegression**. У него есть два метода:
1. **fit** - вычисляет оптимальные веса модели
2. **predict** - классифицирует объект

In [5]:
class LinearRegressionClassifier:
    def fit(self, X, y):
        self.w = np.linalg.inv(X.T @ X) @ X.T @ y

    def predict(self, X):
        return np.array(list(map(lambda x: 1 if x >=0 else 0, X @ self.w)))

    def accuracy(self, y, y_pred):
        print('The accuracy of the classifier is {0:.3f}'.format(accuracy_score(y, y_pred)))

Полезные ссылки:
1. [Если не знаете, что значат слова "Класс" и "ООП"](https://younglinux.info/oopython.php) - достаточно посмотреть первые 3 части, но по желанию можно прочитать и дальше

Точность нашего алгоритма будем определять с помощью функции **accuracy**. Про метрики качества еще поговорим на семинаре.

Теперь мы можем *работать*. Есть 4 этапа:
1. Создать классификатор
2. Обучить его (подобрать веса модели)
3. Сделать предсказания (на отложенной ранее выборке X_test)
4. Посчитать точность классификатора

In [6]:
alg = LinearRegressionClassifier()

In [7]:
alg.fit(X_train, y_train)

In [8]:
y_pred = alg.predict(X_test)

In [9]:
alg.accuracy(y_test, y_pred)

The accuracy of the classifier is 0.762


### SGD

Заново считаем данные (они почти наверняка как-то изменились, когда мы произвели аналитическое решение).  
P.S.: Особенности Python.

In [10]:
cancer = datasets.load_breast_cancer()

In [11]:
X = np.hstack((np.ones(len(cancer.target)).reshape(-1,1),cancer.data))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, cancer.target, random_state=42)

In [13]:
class LinearRegressionClassifierSGD:
    def __init__(self, lr=None):
        if lr is None:
            self.lr = 0.001
        else:
            self.lr = lr

    def dist(self, x, y):
        return np.sqrt(np.sum((x - y) ** 2))

    def fit(self, X, y):
        self.w = np.zeros((X.shape[1],1))
        i = randint(0, X.shape[0] - 1)
        x_selected = X[i]
        y_selected = y[i]
        tmp = self.w - 2 * self.lr * x_selected.reshape(-1,1) *\
              (x_selected @ self.w - y_selected) / X.shape[0]
        while self.dist(self.w, tmp) > 1e-7:
            i = randint(0, X.shape[0] - 1)
            x_selected = X[i]
            y_selected = y[i]
            self.w, tmp = tmp - 2 * self.lr * x_selected.reshape(-1,1) *\
                          (x_selected @ self.w - y_selected) / X.shape[0], self.w

    def predict(self, X):
        return np.array(list(map(lambda x: 1 if x >=0 else 0, X @ self.w)))

    def accuracy(self, y, y_pred):
        print('The accuracy of the classifier is {0:.3f}'.format(accuracy_score(y, y_pred)))

In [14]:
alg = LinearRegressionClassifier()

In [15]:
alg.fit(X_train, y_train)

In [16]:
y_pred = alg.predict(X_test)

In [17]:
alg.accuracy(y_test, y_pred)

The accuracy of the classifier is 0.762


## Boston

**Boston** - датасет о зависимости стоимости домов от некоторых параметров *(их названия можно посмотреть с помощью функций feature_names или почитать о них в DESCR)*. Ваша задача состоит в том, чтобы написать линейный регрессор - вы предполагаете, что зависимость стоимости от заданных параметров линейная.

Начало работы с данными такое же, как и в задаче с классификацией рака.

In [ ]:
boston = datasets.load_boston()

In [ ]:
X = np.hstack((np.ones(len(boston.target)).reshape(-1,1),boston.data))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, boston.target, random_state=42)

### Аналитическое решение

Сейчас нам нужно изменить класс, который был у нас раньше. Действительно, раньше мы решали задачу классификации, а сейчас - регрессии.

In [ ]:
class LinearRegressionRegressor:
    def fit(self, X, y):
        ### YOUR CODE HERE ! ###

    def predict(self, X):
        ### YOUR CODE HERE ! ###
    
    def error(self, y, y_pred):
        print('RMSE is {0:.3f}'.format(np.sqrt(np.sum((y - y_pred) ** 2)) / len(y)))

Вместо метода **accuraccy** у нас теперь метод **error**. О том, почему нельзя оставить **accuracy** и почему **error** выглядит именно так поговорим на семинаре.

In [ ]:
alg = LinearRegressionRegressor()

In [ ]:
alg.fit(X_train, y_train)

In [ ]:
y_pred = alg.predict(X_test)

In [ ]:
alg.error(y_test, y_pred)

### SGD

По аналогии с **LinearRegressionClassifier** допишите **LinearRegressionRegressorSGD**.

In [ ]:
boston = datasets.load_boston()

In [ ]:
X = np.hstack((np.ones(len(boston.target)).reshape(-1,1),boston.data))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, boston.target, random_state=42)

In [ ]:
class LinearRegressionRegressorSGD:
    def __init__(self, lr=None):
        if lr is None:
            self.lr = 0.001
        else:
            self.lr = lr

    def dist(self, x, y):
        return np.sqrt(np.sum((x - y) ** 2))

    def fit(self, X, y):
        ### YOUR CODE HERE ! ###

    def predict(self, X):
        ### YOUR CODE HERE ! ###

    def error(self, y, y_pred):
        print('RMSE is {0:.3f}'.format(np.sqrt(np.sum((y - y_pred) ** 2)) / len(y)))

In [ ]:
alg = LinearRegressionRegressor()

In [ ]:
alg.fit(X_train, y_train)

In [ ]:
y_pred = alg.predict(X_test)

In [ ]:
alg.error(y_test, y_pred)